# Andmete puhastamine

Siin töötleme varasemas etapis kogutud toorandmeid (`toorandmed_aasta.csv`).
Faili leiad projekti ÜHISEST repositooriumist. 

**Sisend:** `../andmed/toorandmed_aasta.csv`

**Väljund:** `../andmed/andmed_aasta.csv`

### Samm 0: Teekide laadimine ja failiteede seadistamine 
Laeme vajalikud Pythoni teegid ja fikseerime sisend- ja väljundfailide asukohad. Loeme sisse puhastamata andmed.

In [80]:
import pandas as pd
import numpy as np
import os

# Konfigureerime Pandase sätteid loetavuse huvides
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

# Sisend- ja väljundfailid
INPUT_FILE = '../andmed/toorandmed_aasta.csv'
OUTPUT_FILE = '../andmed/andmed_aasta.csv'


if os.path.exists(INPUT_FILE):
    df = pd.read_csv(INPUT_FILE, low_memory=False)
    print(f"Toorandmed loetud. Andmestiku suurus: {df.shape[0]} rida, {df.shape[1]} veergu.")
else:
    # Kui faili pole, loome testimiseks tühja DataFrame'i või viskame vea
    raise FileNotFoundError(f"Viga: Sisendfaili '{INPUT_FILE}' ei leitud! Kontrolli failiteed.")

# Kiire pilk andmestruktuurile


print("Samm 1: Teegid laetud ja seadistused tehtud. Andmed sisse loetud.")

df.head(2)

Toorandmed loetud. Andmestiku suurus: 3156 rida, 229 veergu.
Samm 1: Teegid laetud ja seadistused tehtud. Andmed sisse loetud.


,course_uuid,code,title__en,version__title__en,credits,overview__description__en,version__overview__description__en,overview__learning_outcomes_text_en,uuid,state__code,state__et,state__en,last_update,title__et,general__input_languages,general__structural_unit_shares,general__type__code,general__type__et,general__type__en,general__structural_units,additional_info__duration_in_semesters,additional_info__assessment_scale__code,additional_info__assessment_scale__et,additional_info__assessment_scale__en,additional_info__is_vota_course,additional_info__is_continuous_learning_course,overview__description__et,overview__objectives,overview__learning_outcomes,_actions__is_editable,_actions__is_version_creatable,_actions__is_deletable,overview__objectives_text_en,version__uuid,version__code,version__state__code,version__state__et,version__state__en,version__last_update,version__title__et,version__credits,version__credit_type,version__target__year__code,version__target__year__et,version__target__year__en,version__target__semester__code,version__target__semester__et,version__target__semester__en,version__target__study_type__code,version__target__study_type__et,version__target__study_type__en,version__target__language__code,version__target__language__et,version__target__language__en,version__target__part__code,version__target__part__et,version__target__part__en,version__target__faculty__code,version__target__faculty__name__en,version__target__faculty__name__et,version__target__faculty__address,version__target__faculty__city,version__target__faculty__level,version__target__faculty__academic,version__target__faculty__supports_continuous_learning,version__target__course_main_structural_unit__code,version__target__course_main_structural_unit__name__en,version__target__course_main_structural_unit__name__et,version__target__course_main_structural_unit__webpage_url,version__target__course_main_structural_unit__address,version__target__course_main_structural_unit__city,version__target__course_main_structural_unit__street,version__target__course_main_structural_unit__zip,version__target__course_main_structural_unit__level,version__target__course_main_structural_unit__academic,version__target__course_main_structural_unit__supports_continuous_learning,version__target__course_main_structural_unit__parent_code,version__target__label__en,version__target__label__et,version__general__year__code,version__general__year__et,version__general__year__en,version__general__input_languages,version__general__structural_unit_shares,version__general__type__code,version__general__type__et,version__general__type__en,version__additional_info__is_vota_course,version__additional_info__is_continuous_learning_course,version__additional_info__study_levels,version__additional_info__hours__lecture,version__additional_info__hours__practice,version__additional_info__hours__seminar,version__additional_info__hours__colloquium,version__additional_info__hours__web_learning,version__additional_info__hours__independent_work,version__additional_info__hours__individual_work,version__additional_info__hours__internship,version__additional_info__is_enlight_course,version__overview__study_languages,version__overview__description__et,version__overview__objectives,version__overview__learning_outcomes,version__grading__independent_work_assessments,version__grading__grade_preconditions__et,version__grading__grade_evaluation__et,version__grading__debt_elimination__et,version__grading__assessment_scale__code,version__grading__assessment_scale__et,version__grading__assessment_scale__en,version__resources__web_based__code,version__resources__web_based__et,version__resources__web_based__en,version__resources__website_url,version__resources__is_moodle_connected,version__resources__learning_environments,version__resources__materials,version__participants__lecturers,version__schedule__weeks__et,version__schedule__entries,version__parent_uuid,version__parent_code,version__parent_credits,version___action

### Samm 1: eemaldame andmed, mida me ei taha

Rakendame minimaalsed eelfiltrid, mis olid seminaris kokku lepitud:
- eemaldame ained, mille kestus on üle 1 semestri;
- eemaldame kaitsmisega seotud ained (kui pealkiri viitab kaitsmisele);
- proovime jätta ainult päevaõppe, kui vastav tunnus on olemas.

### Samm 2: Puhastatud veergude lisamine

Valime välja huvipakkuvad veerud ning puhastame need. Antud sammu juures seisneb puhastamine õige veeru valimises - juhul kui huvipakkuv sisu võib olla kahes erinevas veerus, eelistame aine konkreetse versiooni infot.


Täpsem kirjeldus aine üldine info vs versiooni info probleemist:
* **Probleem:** Meil on topelt veerge eeldatavasti sama infoga (üldine aine info vs aine konkreetse versiooni info). Nt eestikeelne pealkiri võiks peituda nii `title__et` kui ka `version__title__et` veerus.
* **Lahendus:** Eelistame alati `version__` prefiksiga veerge, sest need kajastavad tegelikku aineprogrammi. Kui see puudub, võtame üldise info.

In [81]:
# --- Samm 1: Eelfiltrid ---
df_work = df.copy()
rows_before = len(df_work)

# 1) eemaldame > 1 semestri ained (kui veerg olemas)
if 'additional_info__duration_in_semesters' in df_work.columns:
    duration = pd.to_numeric(df_work['additional_info__duration_in_semesters'], errors='coerce')
    df_work = df_work[(duration.isna()) | (duration <= 1)]

# 2) eemaldame kaitsmisega seotud ained (heuristika ET+EN pealkirjade põhjal)
title_cols = [c for c in ['version__title__et', 'title__et', 'version__title__en', 'title__en'] if c in df_work.columns]
if title_cols:
    title_text = df_work[title_cols].fillna('').astype(str).agg(' '.join, axis=1)
    df_work = df_work[~title_text.str.contains(r'kaitsm|defen[cs]e', case=False, regex=True)]

# 2b) lisaks eemaldame kaitsmise viited hindamise kirjeldusest (kui olemas)
if 'version__grading__grade_evaluation__et' in df_work.columns:
    eval_text = df_work['version__grading__grade_evaluation__et'].fillna('').astype(str)
    df_work = df_work[~eval_text.str.contains(r'kaitsm', case=False, regex=True)]

# 3) ainult päevaõpe (kui tunnus olemas)
if 'version__target__study_type__et' in df_work.columns:
    study_type = df_work['version__target__study_type__et'].fillna('').astype(str)
    day_mask = study_type.str.contains('päeva', case=False, regex=False)
    if day_mask.any():
        df_work = df_work[day_mask]

print(f"Samm 1: Eelfiltrid rakendatud. Ridasi enne: {rows_before}, pärast: {len(df_work)}")

# --- Samm 2: Võtame ALLES ainult kokkulepitud tunnused ---
selected_raw_cols = [
    'code',
    'credits',
    'additional_info__prerequisites',
    'overview__description__en',
    'overview__description__et',
    'title__en',
    'title__et',
    'version__target__language__et',
    'version__target__semester__et',
    'additional_info__assessment_scale__et',
    'version__overview__description__en',
    'version__overview__description__et',
    'version__overview__study_languages',
    'version__target__course_main_structural_unit__city',
    'version__additional_info__study_levels',
    'version__grading__grade_evaluation__et',
]

# Lisame puuduvad veerud NaN-na, et väljundi skeem oleks stabiilne
for col in selected_raw_cols:
    if col not in df_work.columns:
        df_work[col] = np.nan

def resolve_fields(df):
    """
    Loob uued puhtad veerud, eelistades versiooni-põhist infot.
    Tagastab täiendatud DataFrame'i.
    """

    merge_mapping = [
        ('nimi_et', 'title__et', 'version__title__et'),
        ('nimi_en', 'title__en', 'version__title__en'),
        ('eap', 'credits', 'version__credits'),
        ('kirjeldus', 'overview__description__et', 'version__overview__description__et'),
        ('kirjeldus_en', 'overview__description__en', 'version__overview__description__en'),
    ]

    rename_mapping = [
        ('aine_kood', 'code'),
        ('semester', 'version__target__semester__et'),
        ('oppejoud_json', 'version__participants__lecturers'),
        ('toimumisajad_json', 'version__schedule__entries'),
    ]

    print('Alustan veergude filtreerimist ja ühendamist...')

    for new_col, base, version in merge_mapping:
        base_exists = base in df.columns if base else False
        ver_exists = version in df.columns if version else False

        if base_exists and ver_exists:
            df[new_col] = df[version].fillna(df[base])
        elif ver_exists:
            df[new_col] = df[version]
        elif base_exists:
            df[new_col] = df[base]
        else:
            df[new_col] = np.nan

    for new_col, source in rename_mapping:
        if source in df.columns:
            df[new_col] = df[source]
        else:
            df[new_col] = np.nan

    return df

# Rakendame funktsiooni
df_resolved = resolve_fields(df_work.copy())

# Kontrollime tulemust
print('Samm 2: Puhastatud veergude lisamine tehtud.')
print(df_resolved[['aine_kood', 'semester']].isnull().sum())
print('Semestrid:', sorted([str(x) for x in df_resolved['semester'].dropna().unique()]))
print('Valitud veerud (nõutud 16):', len(selected_raw_cols))
display(pd.Series(selected_raw_cols, name='selected_features'))

Samm 1: Eelfiltrid rakendatud. Ridasi enne: 3156, pärast: 3058
Alustan veergude filtreerimist ja ühendamist...
Samm 2: Puhastatud veergude lisamine tehtud.
aine_kood    0
semester     0
dtype: int64
Semestrid: ['kevad', 'sügis']
Valitud veerud (nõutud 16): 16


0                                                   code
1                                                credits
2                         additional_info__prerequisites
3                              overview__description__en
4                              overview__description__et
5                                              title__en
6                                              title__et
7                          version__target__language__et
8                          version__target__semester__et
9                  additional_info__assessment_scale__et
10                    version__overview__description__en
11                    version__overview__description__et
12                    version__overview__study_languages
13    version__target__course_main_structural_unit__city
14                version__additional_info__study_levels
15                version__grading__grade_evaluation__et
Name: selected_features, dtype: str


### Samm 3: JSON väljade parsimine

See koodiplokk tegeleb tehniliste JSON-formaadis veergude teisendamisega inimloetavaks tekstiks. Eesmärk on eraldada keerulisest struktuurist vaid oluline sisu (nt nimed või nädalapäevad) ja vormistada see komadega eraldatud loeteluks.

Näide teisendamisest (õppejõud):
* Enne (Toores JSON): ``'[{"person_name": "Mari Mets", "id": 101}, {"person_name": "Jaan Kask", "id": 102}]'``
* Pärast (Puhastatud): ``"Jaan Kask, Mari Mets"``

In [90]:
# Samm 3: JSON-väljade parsimine + ühildatud väljad
import json

def parse_json_safe(value):
    if value is None or (isinstance(value, float) and pd.isna(value)):
        return None
    if isinstance(value, (list, dict)):
        return value
    text = str(value).strip()
    if text == '' or text.lower() == 'nan':
        return None
    try:
        return json.loads(text)
    except (json.JSONDecodeError, TypeError):
        return None

def unique_join(values, sep=', '):
    seen = []
    for val in values:
        if val and val not in seen:
            seen.append(val)
    return sep.join(seen) if seen else np.nan

def extract_enum_list_et(value):
    data = parse_json_safe(value)
    if data is None:
        return value if (pd.notna(value) and str(value).strip() != '') else np.nan
    if isinstance(data, dict):
        data = [data]

    out = []
    for item in data if isinstance(data, list) else []:
        if isinstance(item, str):
            text = item.strip()
            if text:
                out.append(text)
        elif isinstance(item, dict):
            candidate = (
                item.get('et')
                or item.get('name__et')
                or item.get('name')
                or item.get('title')
                or item.get('en')
                or item.get('code')
            )
            if candidate and str(candidate).strip():
                out.append(str(candidate).strip())

    return unique_join(out)

def extract_prerequisites(value):
    data = parse_json_safe(value)
    if data is None:
        return value if (pd.notna(value) and str(value).strip() != '') else np.nan

    def item_to_text(item):
        if isinstance(item, str):
            return item.strip() if item.strip() else None
        if isinstance(item, dict):
            code = item.get('code') or item.get('course_code')
            name = (
                item.get('name__et')
                or item.get('title__et')
                or item.get('name')
                or item.get('title')
            )
            if code and name:
                return f"{code} ({name})"
            if code:
                return str(code).strip()
            if name:
                return str(name).strip()
        return None

    if isinstance(data, dict):
        if 'and' in data and isinstance(data['and'], list):
            parts = [item_to_text(x) for x in data['and']]
            return unique_join([p for p in parts if p], sep=' JA ')
        if 'or' in data and isinstance(data['or'], list):
            parts = [item_to_text(x) for x in data['or']]
            return unique_join([p for p in parts if p], sep=' VÕI ')
        single = item_to_text(data)
        return single if single else np.nan

    if isinstance(data, list):
        parts = [item_to_text(x) for x in data]
        return unique_join([p for p in parts if p], sep=' JA ')

    return np.nan

# JSON-väljade puhastus
if 'additional_info__prerequisites' in df_resolved.columns:
    df_resolved['eeldusained'] = df_resolved['additional_info__prerequisites'].apply(extract_prerequisites)
else:
    df_resolved['eeldusained'] = np.nan

if 'version__overview__study_languages' in df_resolved.columns:
    df_resolved['oppekeeled'] = df_resolved['version__overview__study_languages'].apply(extract_enum_list_et)
else:
    df_resolved['oppekeeled'] = np.nan

if 'version__additional_info__study_levels' in df_resolved.columns:
    df_resolved['oppeaste'] = df_resolved['version__additional_info__study_levels'].apply(extract_enum_list_et)
else:
    df_resolved['oppeaste'] = np.nan

# Üks ühine õppekeele väli: eelistame detailset oppekeeli, fallback target-keelele
lang_target = df_resolved['version__target__language__et'].fillna('') if 'version__target__language__et' in df_resolved.columns else ''
lang_detail = df_resolved['oppekeeled'].fillna('') if 'oppekeeled' in df_resolved.columns else ''
df_resolved['oppekeel'] = lang_detail.where(lang_detail.str.strip() != '', lang_target)

# Ühildame description väljad (version eelistatud, duplikaate ei jäta lõppväljundisse)
desc_et_ver = df_resolved['version__overview__description__et'].fillna('')
desc_et_base = df_resolved['overview__description__et'].fillna('')
desc_en_ver = df_resolved['version__overview__description__en'].fillna('')
desc_en_base = df_resolved['overview__description__en'].fillna('')

df_resolved['kirjeldus_et'] = desc_et_ver.where(desc_et_ver.str.strip() != '', desc_et_base)
df_resolved['kirjeldus_en'] = desc_en_ver.where(desc_en_ver.str.strip() != '', desc_en_base)
df_resolved['kirjeldus'] = (
    'ET: ' + df_resolved['kirjeldus_et'].fillna('').astype(str).str.strip()
    + '\nEN: ' + df_resolved['kirjeldus_en'].fillna('').astype(str).str.strip()
).str.strip()

print('Samm 3: JSON-väljad parsitud ja ühildatud väljad loodud.')
display(df_resolved[['aine_kood', 'eeldusained', 'oppekeel', 'oppeaste', 'kirjeldus']].head(3))

Samm 3: JSON-väljad parsitud ja ühildatud väljad loodud.


,aine_kood,eeldusained,oppekeel,oppeaste,kirjeldus
0,OIEO.06.046,NaN,eesti keel,magistriõpe,"ET: Kursuse raames käsitletakse rahvusvahelise eraõiguse põhiteemasid, terminoloogiat ja kujunem..."
1,KKSB.05.092,NaN,eesti keel,bakalaureuseõpe,ET: Aines Praktiseerimine töökeskkonnas laiendab üliõpilane oma silmaringi füsioteraapia valdkon...
2,ARKI.02.030,"AR00.00.016 ({'en': 'Fundamentals of Diagnostics', 'et': 'Diagnostika alused'})",inglise keel,integreeritud bakalaureuse- ja magistriõpe,"ET: Lastekirurgiliste haiguste diagnoosimise ja ravi printsiibid, metoodika ja taktika. Antud ai..."


### Samm 4: Väljadele paremad nimed (esitlusvaade)

Selles sammus anname tehnilistele veerunimedele inimloetavad nimed. 
Oluline: algseid veerunimesid ei muudeta, et väljund-CSV jääks kokkulepitud skeemiga.

In [87]:
# Samm 4: Loome esitlusvaate jaoks inimloetavad väljanimed (ühildatud väljadele)
pretty_name_map = {
    'aine_kood': 'Aine kood',
    'nimi_et': 'Aine nimetus (ET)',
    'nimi_en': 'Aine nimetus (EN)',
    'eap': 'EAP',
    'semester': 'Semester',
    'oppekeel': 'Õppekeel',
    'oppeaste': 'Õppeaste',
    'version__target__course_main_structural_unit__city': 'Linn',
    'additional_info__assessment_scale__et': 'Hindamisskaala',
    'version__grading__grade_evaluation__et': 'Hindamise kirjeldus',
    'eeldusained': 'Eeldusained',
    'kirjeldus': 'Kirjeldus',
}

df_pretty = df_resolved.rename(columns=pretty_name_map).copy()

print('Samm 4: Inimloetavad väljanimed lisatud (duplikaate ei kuvata).')
display(pd.Series(pretty_name_map, name='pretty_field_names'))
print('\nEelvaade ilusate väljanimedega:')
display(df_pretty[[c for c in pretty_name_map.values() if c in df_pretty.columns]].head(2))

Samm 4: Inimloetavad väljanimed lisatud (duplikaate ei kuvata).


aine_kood                                                       Aine kood
nimi_et                                                 Aine nimetus (ET)
nimi_en                                                 Aine nimetus (EN)
eap                                                                   EAP
semester                                                         Semester
oppekeel                                                         Õppekeel
oppeaste                                                         Õppeaste
version__target__course_main_structural_unit__city                   Linn
additional_info__assessment_scale__et                      Hindamisskaala
version__grading__grade_evaluation__et                Hindamise kirjeldus
eeldusained                                                   Eeldusained
kirjeldus                                                       Kirjeldus
Name: pretty_field_names, dtype: str


Eelvaade ilusate väljanimedega:


,Aine kood,Aine nimetus (ET),Aine nimetus (EN),EAP,Semester,Õppeaste,Linn,Hindamisskaala,Hindamise kirjeldus,Eeldusained,Kirjeldus
0,OIEO.06.046,Rahvusvaheline eraõigus,Private International Law,6.0,kevad,magistriõpe,Tartu linn,"Eristav (A, B, C, D, E, F, mi)",100% kirjalik eksam.,NaN,"ET: Kursuse raames käsitletakse rahvusvahelise eraõiguse põhiteemasid, terminoloogiat ja kujunem..."
1,KKSB.05.092,Praktiseerimine töökeskkonnas,Practice in the Work Environment,3.0,kevad,bakalaureuseõpe,Tartu linn,"Eristamata (arv, m.arv, mi)",NaN,NaN,ET: Aines Praktiseerimine töökeskkonnas laiendab üliõpilane oma silmaringi füsioteraapia valdkon...


### Samm 5: Lõplik viimistlus

Valime lõplikud veerud, lisame hard-filtri tunnused ja valmistame RAG jaoks ühendatud tekstivälja.

In [93]:
# Samm 5: Lõplik viimistlus (ilma overview/version duplikaatideta)
final_cols = [
    'aine_kood', 'nimi_et', 'nimi_en', 'eap', 'semester',
    'oppekeel', 'oppeaste',
    'version__target__course_main_structural_unit__city',
    'additional_info__assessment_scale__et', 'version__grading__grade_evaluation__et',
    'eeldusained', 'kirjeldus',
]

existing_cols = [c for c in final_cols if c in df_resolved.columns]
df_final = df_resolved[existing_cols].copy()

# Aine kood peab olema unikaalne ID: korduvatele lisame sufiksi (_v2, _v3, ...)
base_code = df_final['aine_kood'].fillna('PUUDUB').astype(str).str.strip()
base_code = base_code.mask(base_code == '', 'PUUDUB')
occurrence = base_code.groupby(base_code).cumcount() + 1
df_final['aine_kood'] = np.where(
    occurrence.eq(1),
    base_code,
    base_code + '_v' + occurrence.astype(str)
)

duplicate_rows = int((occurrence > 1).sum())
print(f"Lõplik ridade arv: {len(df_final)}, lõplik veergude arv: {len(df_final.columns)}")
print('Samm 5: Lõplik andmestik koostatud.')
print(f"Korduvaid ainekoode, millele lisati suffix: {duplicate_rows}")
print('Kas aine_kood on unikaalne?', df_final['aine_kood'].is_unique)
print('Lõppveerud (ühildatud):')
display(pd.Series(df_final.columns, name='final_columns'))

Lõplik ridade arv: 3058, lõplik veergude arv: 12
Samm 5: Lõplik andmestik koostatud.
Korduvaid ainekoode, millele lisati suffix: 165
Kas aine_kood on unikaalne? True
Lõppveerud (ühildatud):


0                                              aine_kood
1                                                nimi_et
2                                                nimi_en
3                                                    eap
4                                               semester
5                                               oppekeel
6                                               oppeaste
7     version__target__course_main_structural_unit__city
8                  additional_info__assessment_scale__et
9                 version__grading__grade_evaluation__et
10                                           eeldusained
11                                             kirjeldus
Name: final_columns, dtype: str

### Samm 6: Salvestamine ja kodutöö väljundid

Selles sammus näitame kodutöö esitamiseks vajalikud väljundid:
- puuduvate väärtuste hulk igas veerus;
- kategooriliste tunnuste enim levinud väärtused;
- "Sissejuhatus andmeteadusesse" aine puhastatud kujul loetavalt;
- tunnus `kirjeldus`, mis ühendab ühe aine koguinfo tekstiks;
- tunnuse `kirjeldus` tähemärkide arvu statistika.

In [94]:


# Pretty nimede kaart ainult kuvamiseks (CSV jääb tehniliste nimedega)
pretty_map = pretty_name_map if 'pretty_name_map' in globals() else {c: c for c in df_final.columns}
df_final_pretty = df_final.rename(columns=pretty_map).copy()

# 1) Puuduvate väärtuste hulk igas veerus
print('\nPuuduvate väärtuste arv veergude kaupa:')
missing = df_final.isna().sum().sort_values(ascending=False)
missing_pretty = missing.copy()
missing_pretty.index = [pretty_map.get(col, col) for col in missing_pretty.index]
display(missing_pretty)

# 2) Kategooriliste tunnuste enim levinud väärtused
print('\nKategooriliste tunnuste top-väärtused:')
categorical_cols = [
    'oppekeel',
    'semester',
    'oppeaste',
    'version__target__course_main_structural_unit__city',
    'additional_info__assessment_scale__et',
]

for col in categorical_cols:
    if col in df_final.columns:
        pretty_col = pretty_map.get(col, col)
        print(f"\nVeerg: {pretty_col}")
        top_vals = df_final[col].value_counts(dropna=False).head(5)
        top_vals.name = pretty_col
        display(top_vals)

# 3) Näita AINULT ainet koodiga LTAT.02.002
target_code = 'LTAT.02.002'
sample_df = df_final[df_final['aine_kood'].astype(str).str.strip().eq(target_code)].copy()

if not sample_df.empty:
    sample_pretty = sample_df.head(1).T
    sample_pretty.index = [pretty_map.get(idx, idx) for idx in sample_pretty.index]
    sample_pretty.columns = ['väärtus']
    print(f'\nNäidis: aine koodiga {target_code} (puhastatud kuju)')
    display(sample_pretty)
else:
    print(f'\nAinet koodiga {target_code} ei leitud andmestikust.')

# 4) Lisame tunnuse 'kirjeldus_kogu' (kogu aineinfo ühes tekstis)
df_report = df_final.copy()
report_cols = [c for c in df_report.columns if c != 'kirjeldus']

def build_course_description(row):
    parts = []
    for col in report_cols:
        value = row.get(col)
        if pd.notna(value) and str(value).strip() != '':
            parts.append(f"{pretty_map.get(col, col)}: {value}")
    if pd.notna(row.get('kirjeldus')) and str(row.get('kirjeldus')).strip() != '':
        parts.append(f"{pretty_map.get('kirjeldus', 'kirjeldus')}: {row.get('kirjeldus')}")
    return "\n".join(parts)

df_report['kirjeldus_kogu'] = df_report.apply(build_course_description, axis=1)
df_report['kirjeldus_pikkus'] = df_report['kirjeldus_kogu'].str.len()

print('\nTunnuse kirjeldus pikkuse statistika:')
display(df_report['kirjeldus_pikkus'].describe())
print(f"\nKokku tähemärke: {df_report['kirjeldus_pikkus'].sum()}")

# 5) Kiire eelvaade pretty päistega
print('\nLõppandmestiku eelvaade (pretty päised):')
display(df_final_pretty.head(3))

# Salvesta puhastatud andmestik
df_final.to_csv(OUTPUT_FILE, index=False)
print(f"Samm 6: Andmestik salvestatud faili '{OUTPUT_FILE}'.")


Puuduvate väärtuste arv veergude kaupa:


Eeldusained            2506
Linn                   1128
Hindamise kirjeldus     369
Õppeaste                 38
EAP                       0
Aine nimetus (EN)         0
Aine nimetus (ET)         0
Aine kood                 0
Semester                  0
Õppekeel                  0
Hindamisskaala            0
Kirjeldus                 0
dtype: int64


Kategooriliste tunnuste top-väärtused:

Veerg: Õppekeel


oppekeel
eesti keel                  1929
inglise keel                 768
eesti keel, inglise keel     113
vene keel                     48
inglise keel, eesti keel      44
Name: Õppekeel, dtype: int64


Veerg: Semester


...


Tunnuse kirjeldus pikkuse statistika:


count    3058.000000
mean     1380.362328
std       695.969214
min       275.000000
25%       894.250000
50%      1221.500000
75%      1673.000000
max      5708.000000
Name: kirjeldus_pikkus, dtype: float64


Kokku tähemärke: 4221148

Lõppandmestiku eelvaade (pretty päised):


,Aine kood,Aine nimetus (ET),Aine nimetus (EN),EAP,Semester,Õppekeel,Õppeaste,Linn,Hindamisskaala,Hindamise kirjeldus,Eeldusained,Kirjeldus
0,OIEO.06.046,Rahvusvaheline eraõigus,Private International Law,6.00,kevad,eesti keel,magistriõpe,Tartu linn,"Eristav (A, B, C, D, E, F, mi)",100% kirjalik eksam.,NaN,"ET: Kursuse raames käsitletakse rahvusvahelise eraõiguse põhiteemasid, terminoloogiat ja kujunem..."
1,KKSB.05.092,Praktiseerimine töökeskkonnas,Practice in the Work Environment,3.00,kevad,eesti keel,bakalaureuseõpe,Tartu linn,"Eristamata (arv, m.arv, mi)",NaN,NaN,ET: Aines Praktiseerimine töökeskkonnas laiendab üliõpilane oma silmaringi füsioteraapia valdkon...
2,ARKI.02.030,Lastekirurgia,Paediatric Surgery,2.25,sügis,inglise keel,integreeritud bakalaureuse- ja magistriõpe,Tartu linn,"Eristamata (arv, m.arv, mi)","A part of the complec exam ""Surgery""","AR00.00.016 ({'en': 'Fundamentals of Diagnostics', 'et': 'Diagnostika alused'})","ET: Lastekirurgiliste haiguste diagnoosimise ja ravi printsiibid, metoodika ja taktika. Antud ai..."


Samm 6: Andmestik salvestatud faili '../andmed/andmed_aasta.csv'.
